# 💿 Start een audio opname


In [35]:
# importeert modules
import pyaudio
import wave
import os
import numpy as np

# vaste meetgegevens
table = 12     # Welke tafel je meet (zelf genummerd), dit ga je dus steeds varieren      tussen [1-26]
index = 1     # De index voor welke frequentie we hebben gebruikt | 1 = 164hz, 2 = 276hz     tussen [1-2]

# Audioconfiguratie
Chunk = 1024      # Aantal frames per buffer (verkleint geheugenbelasting)
Format = pyaudio.paInt16      # 16-bit audioformaat (standaard voor veel toepassingen)
Rate = 44100     # Samplefrequentie in Hz (CD-kwaliteit)

seconds =  5     # Totale opnameduur in seconden
aantal_metingen = 5     # aantal metingen die je in een keer wilt uitvoeren

folder_locatie = 'AudioFiles'     # De naam van de folder waar de metingen worden opgeslagen

def GenerateFileName(counter):
    return f"{folder_locatie}/Meting_RT60_{index}_{table}_{counter}.wav" # zorg voor een duidelijke naamgeving.

for poging in range(aantal_metingen):
    filename = GenerateFileName(poging)
    print(f"Opname van bestand: {filename}")
    
    # Controleer of het bestand al bestaat en waarschuw de gebruiker
    if os.path.exists(filename):
        print(f"⚠️ Waarschuwing: {filename} bestaat al en zal worden overschreven.")
    
    # Initialiseer PyAudio en start opname
    p = pyaudio.PyAudio()
    stream = p.open(format = Format, channels = 1, rate = Rate, input = True, frames_per_buffer = Chunk)
    
    print("🎤 Start opname...")
    frames = []       # Opslag voor audioblokken
    clipping_detected = False       # Vlag voor detectie van oversturing
    
    # Leest de audioblokken en slaat ze op
    for _ in range(int(Rate / Chunk * seconds)):
        data = stream.read(Chunk)
        frames.append(data)

        # Controleer op clipping (oversturing)
        audio_array = np.frombuffer(data, dtype=np.int16)
        if np.any(audio_array >= 32767) or np.any(audio_array <= -32768):
            clipping_detected = True
    
    print("🛑 Opname gestopt.")
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Opslaan als WAV-bestand
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(1)    # Mono-opname
        wf.setsampwidth(p.get_sample_size(Format))
        wf.setframerate(Rate)
        wf.writeframes(b''.join(frames))

   # Geef feedback over mogelijke vervorming
    if clipping_detected:
        print("⚠️ Waarschuwing: Clipping gedetecteerd! Het opgenomen geluid is mogelijk vervormd.")
    else:
        print("✅ Geen clipping gedetecteerd.")



# Verwerking van audio bestand
### Berekend nagalmtijd / maakt geluidssterkte tegen tijd grafieken

In [37]:
# importeert modules
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os

# Vaste meetgegevens
I_0 = 1e-1      # Dit de referentiegeluidsintensiteit
epsilon = 1e-10     # Een hele kleine waarde zodat een logaritmische functie geen 0 raakt en een error veroorzaakt

tijdsinterval = 0.1     # De waarde waarmee de grafiek meer geleidelijk wordt. Hoe hoger het getal hoe geleidelijker de grafiek
dB_afname = 18      # Het dB verschil waartussen je de nagalmtijd wil meten (dit is standaart 60dB afname)

# Gegevens om de eerste en de laatste ... seconden van de meting af te snijden.
afwijking_begin = 0.5
afwijking_eind = 0.5

folder_locatie = 'AudioFiles'

# Een functie die de grafiek inkort met de bovenste 2 gegevens
def ShortenFunction(time,loudness):
    a = int(len(time) * afwijking_begin / max(time))
    b = int(len(time) * (max(time) - afwijking_eind) / max(time) )

    return time[a : b], loudness[a : b]

# Een functie om de nagalmtijd te berekenen
def CalculateRT60(time,loudness):
    # De piek van de grafiek & de plek waar het aantal dB met ... is gedaald na de piek
    max_L = np.max(loudness)
    target_L = max_L - dB_afname

    max_t = ...
    target_t = max(time)

    debounce = True

    for i in range(np.argmax(loudness),len(loudness)):
        
        # Dit zorgt ervoor dat de maximale L het einde van een lange piek pakt met correctiefout 5
        if loudness[i] <= max_L - 3 and debounce:
            debounce = False
            max_t = time[i]

        # Zoekt het punt waar het aantal dB vanaf de piek ... dB afneemt.
        if loudness[i] <= target_L:   # Berekend de plek waar het aantal dB met ... is gedaald
            target_t = time[i]
            break

    # Berekent de nagalmtijd
    RT60 = (target_t - max_t) * (60 / (max_L - target_L))
    
    return RT60, {
        'max_L': max_L,
        'target_L': target_L,
        'max_t': max_t,
        'target_t': target_t,
    }

# Maakt een grafieken van de bewerkte data's
def SketchGraph(t,L,RT60,data):
    plt.figure(figsize=(6, 3))
    plt.plot(t, L)

    # verticale lijn door het maximum en het gemiddelde
    plt.axhline(data['max_L'], color='green', linestyle='--', label='Max dB')
    plt.axhline(data['target_L'], color='red', linestyle='--', label='Gem dB')

    # horizontale lijn door het maximum en het gemiddelde
    plt.axvline(data['max_t'], color='green', linestyle='--')
    plt.axvline(data['target_t'], color='red', linestyle='--')

    plt.xlabel("Tijd (s)")
    plt.ylabel("Geluidsterkte (dB)")

    plt.grid(True)
    plt.legend(loc='lower center')
    
    plt.title(f"Geluidsterkte tegen de tijd - {data['title']} - {data['frequency']}hz\n\n• nagalmtijd: {RT60:.2f}s •")
    plt.show()

# Leest alle audio files
def ReadFiles1():
    for frequency in range(1,3):
        # Naamgeving frequentie
        rl_frequency = 0
        if frequency == 1: rl_frequency = 164
        if frequency == 2: rl_frequency = 276
        
        for table in range(1,27):
            for attempt in range(5):
                # File zoeken
                file_name = f'{folder_locatie}/meting_nagalmtijd_{table}_{frequency}_{attempt}.wav'
                if not os.path.exists(file_name): continue
                    
                # Data verwerken van de audio files
                sample_rate, audio_data = wavfile.read(file_name) # frequentie van de audio & het aantal metingen in (W/m2)
                audio_duration = len(audio_data) / sample_rate

                t = np.linspace(0, audio_duration, len(audio_data)) # tijd (s)
                L = 20 * np.log10(np.abs(audio_data) / I_0 + epsilon) # geluidssterkte (dB)  

                # Maakt een geleidelijkere lijn van L tegen tijd
                window_size = int(tijdsinterval * sample_rate)
                avg_L = np.convolve(np.abs(L), np.ones(window_size) / window_size, mode='same') # gemiddelde van L

                # Nagalmtijd berekenen en gegevens terugbrengen
                t,L = ShortenFunction(t,avg_L)
                RT60, data = CalculateRT60(t,L)
                
                # Grafiek maken
                data['title'] = f'Tafel {table} (nr.{attempt+1})'
                data['frequency'] = rl_frequency

                SketchGraph(t,L,RT60,data)

# Roep de functie aan om de gegevens te verwerken
ReadFiles1()

# 🗺️ Genereert heatmap
#### Dit is een handmatige code! 


In [39]:
# impoort module voor heatmap generatie
import seaborn as sns
import matplotlib.pyplot as plt

handmatige_nagalmtijden = {
    '164hz': [
        [0.78, 0.66, 0.65, 0.49, 0.73, 0.69],    # Rij 1
        [0.81, 0.56, 0.56, 0.53, 0.64, 0.73],    # Rij 2
        [0.67, 0.58, 0.52, 0.63, 0.64, 0.60],    # Rij 3
        [0.45, 0.45, 0.55, 0.57, 0.45, 0.48],    # Rij 4
        [np.nan, np.nan, 0.63, 0.55, np.nan, np.nan],    # Rij 5
    ],
    '276hz': [
        [0.66, 0.58, 0.64, 0.47, 0.64, 0.65],    # Rij 1
        [0.51, 0.67, 0.73, 0.41, 0.65, 0.48],    # Rij 2
        [0.60, 0.53, 0.65, 0.74, 0.47, 0.66],    # Rij 3
        [0.69, 0.64, 0.58, 0.65, 0.58, 0.75],    # Rij 4
        [np.nan, np.nan, 0.50, 0.52, np.nan, np.nan],    # Rij 5          
    ]
}

# Maakt een heatmap voor de 2 verschillende frequenties
def CreateHeatmap(frequency):
    nagalmtijden = np.array(handmatige_nagalmtijden[frequency])
    labels = ['Rij 1', 'Rij 2', 'Rij 3', 'Rij 4', 'Rij 5']
    
    sns.heatmap(nagalmtijden, annot=True, cmap='coolwarm', cbar=True, fmt='.2f', linewidths=5,xticklabels=0, yticklabels=labels)

    plt.title(f'Heatmap klaslokaal {frequency}')
    plt.show()

CreateHeatmap('164hz')
CreateHeatmap('276hz')